In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine
from sqlalchemy.types import JSON
from pandas.io.json import json_normalize
import sqlite3
import numpy as np

In [ ]:
# Convert bundle_data.json to DB
df = pd.DataFrame()

f = open('bundle_data.json', 'r', encoding='utf-8')
l = f.readlines()
for line in l:
    a = eval(line)
    j = json.dumps(a)
    df = df.append(pd.read_json(j))
    #print(j)

f.close()

#engine = create_engine('sqlite:///steam_data.db')
#df.to_sql('bundle_data', engine, dtype={'items': JSON})

In [ ]:
# Convert bundle_data.json to DB v2

dlist = []
items_dict = {}
i = 0
f = open('bundle_data.json', 'r', encoding='utf-8')
l = f.readlines()
for line in l:
    a = eval(line)
    dlist.append(a)
    
    e = eval(str(a['items']))

    for row in e:
        items_dict[i] = {"bundle_final_price": a['bundle_final_price'], "bundle_url": a['bundle_url'], "bundle_price": a['bundle_price'], "bundle_name": a['bundle_name'], "bundle_id": a['bundle_id'], "genre":row['genre'], "item_id":row['item_id'], "discounted_price":row['discounted_price'], "item_url":row['item_url'], "item_name":row['item_name']}
        i = i+1

f.close()

df_output = pd.DataFrame.from_dict(items_dict, "index")
df_output

engine = create_engine('sqlite:///steam_data.db')
df_output.to_sql('bundle_data', engine)

In [ ]:
# Convert stram_games.json to DB
df1 = pd.DataFrame()

f = open('steam_games.json', 'r', encoding='utf-8')
l = f.readlines()
for line in l:
    a = eval(line)
    df1 = df1.append(json_normalize(a))

f.close()

df1.to_sql('steam_games', engine, dtype={'genres': JSON, 'specs': JSON, 'tags': JSON})

In [5]:
# Convert australian_users_items.json to DB

f = open('australian_users_items.json', 'r', encoding='utf-8')
data_list = []
l = f.readlines()
limit = 5000
i = 0
for line in l:
    a = eval(line)
    data_list.append(a)
    i = i+1
    if i>limit:
        break

f.close()

df3=pd.DataFrame(data_list)

dict = {}
i = 0
for index, line in df3.iterrows():
    s = str(line.at['items'])
    e = eval(s)
    df_temp = pd.DataFrame(e)

    for index, row in df_temp.iterrows():
        # add a dictionary entry to the final dictionary
        dict[i] = {"user_id": line.user_id, "items_count": line.items_count, "steam_id": line.steam_id, "user_url": line.user_url, "item_id": row.item_id, "item_name":row.item_name, "playtime_forever":row.playtime_forever, "playtime_2weeks":row.playtime_2weeks}
        i = i + 1

df_output = pd.DataFrame.from_dict(dict, "index")

engine = create_engine('sqlite:///steam_data.sqlite')
df_output.to_sql('australian_users_items', engine)



In [1]:
# Fetching data
#Connecting to database

path = "../input/"  #Insert path here
database = 'steam_data.sqlite'
conn = sqlite3.connect(database)

In [4]:
#Explorer genres in steam_games table

game_genres = pd.read_sql("select steam_games.genres from steam_games;", conn)

genres_set = set()

for index, row in game_genres.iterrows():
    if str(row.at['genres']) != 'null':
        r = eval(str(row.at['genres']))
        for i in r:
            genres_set.add(i)

print(genres_set)

{'Utilities', 'Audio Production', 'Photo Editing', 'Accounting', 'Casual', 'Racing', 'Action', 'Software Training', 'Web Publishing', 'Sports', 'Education', 'Simulation', 'Free to Play', 'RPG', 'Animation &amp; Modeling', 'Video Production', 'Design &amp; Illustration', 'Early Access', 'Adventure', 'Indie', 'Massively Multiplayer', 'Strategy'}


In [40]:
#Explorer specs in steam_games table

game_specs = pd.read_sql("select steam_games.specs from steam_games;", conn)

specs_set = set()

for index, row in game_specs.iterrows():
    if str(row.at['specs']) != 'null':
        r = eval(str(row.at['specs']))
        for i in r:
            specs_set.add(i)

print(specs_set)

{'Co-op', 'Cross-Platform Multiplayer', 'Keyboard / Mouse', 'Mods', 'Local Multi-Player', 'Multi-player', 'Room-Scale', 'Single-player', 'Oculus Rift', 'Tracked Motion Controllers', 'Local Co-op', 'Online Co-op', 'Steam Workshop', 'Full controller support', 'Steam Cloud', 'Partial Controller Support', 'SteamVR Collectibles', 'Stats', 'Steam Trading Cards', 'Seated', 'Game demo', 'HTC Vive', 'In-App Purchases', 'Gamepad', 'MMO', 'Steam Turn Notifications', 'Windows Mixed Reality', 'Downloadable Content', 'Shared/Split Screen', 'Online Multi-Player', 'Mods (require HL2)', 'Captions available', 'Steam Leaderboards', 'Valve Anti-Cheat enabled', 'Steam Achievements', 'Includes Source SDK', 'Includes level editor', 'Mods (require HL1)', 'Standing', 'Commentary available'}


In [41]:
#Explorer tags in steam_games table

game_tags = pd.read_sql("select steam_games.tags from steam_games;", conn)

tags_set = set()

for index, row in game_tags.iterrows():
    if str(row.at['tags']) != 'null':
        r = eval(str(row.at['tags']))
        for i in r:
            tags_set.add(i)

print(tags_set)

{'Magic', 'Investigation', 'Isometric', 'Rhythm', '4 Player Local', '4X', 'Female Protagonist', 'Animation & Modeling', 'Driving', '3D Platformer', 'Cartoon', 'Turn-Based Combat', 'Multiplayer', 'Satire', 'Utilities', 'Underwater', 'Board Game', 'Card Game', 'Trading Card Game', 'Capitalism', 'Philisophical', 'TrackIR', 'Dungeon Crawler', 'Supernatural', 'Hack and Slash', 'Crafting', 'Web Publishing', 'Dark Fantasy', 'Singleplayer', 'Time Travel', 'Mining', 'Top-Down', 'Military', 'Parkour', 'Mouse only', 'Modern', 'Hunting', 'Classic', 'RPG', 'Documentary', 'Gore', 'Free to Play', 'Artificial Intelligence', 'Survival Horror', 'Interactive Fiction', 'Psychological', 'Noir', '2D Fighter', 'Action', 'CRPG', 'Online Co-Op', 'Fantasy', 'Cartoony', 'Side Scroller', 'MOBA', 'Character Customization', 'Nonlinear', 'Cute', 'Dragons', 'Rogue-like', 'Agriculture', 'Turn-Based', 'Pool', 'Education', 'Party-Based RPG', '6DOF', 'Star Wars', 'Puzzle', 'Great Soundtrack', 'Early Access', 'Time Manipu

In [2]:
# Convert genres into binary format using one-hot encoding, then insert into database
from sklearn.preprocessing import MultiLabelBinarizer

#Fetching required data tables
user_game_genres = pd.read_sql("select id, genres from steam_games where genres is not null and genres <>'null';", conn)

#Encode genres
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(user_game_genres['genres'].apply(eval)), columns=mlb.classes_)
result = pd.concat([user_game_genres['id'], encoded], axis=1)

result

#Insert into DB
engine = create_engine('sqlite:///steam_data.sqlite')
result.to_sql('item_genres', engine)

,id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,761140,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,643980,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,670290,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,767400,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,772540,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28847,745400,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28848,773640,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
28849,733530,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28850,610660,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
